<a href="https://colab.research.google.com/github/jmestanza/natural-language-processing-practice/blob/main/desafios/Desafio_2/Desafio_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Crear sus propios vectores con Gensim
basado en lo visto en clase con otro
dataset.
Probar términos de interés y explicar
similitudes en el espacio de embeddings.
Intentar plantear y probar tests de
analogías. Graficar los embeddings
resultantes.
Sacar conclusiones.

Algunos de los sitios de textos sugeridos fueron:
- [Project Gutenberg](https://www.gutenberg.org/browse/scores/top)
- [Textos.info](https://www.textos.info/)

Se eligió analizar los textos pertenecientes a [The Project Gutenberg eBook of The Adventures of Sherlock Holmes](https://www.gutenberg.org/cache/epub/1661/pg1661.txt)


Por razones de compatibilidad con gensim, pandas y numpy hay que correr la siguiente linea de código.

In [1]:
!pip uninstall -y numpy gensim pandas
!pip install --upgrade numpy gensim pandas==2.2.2

Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
Found existing installation: gensim 4.3.3
Uninstalling gensim-4.3.3:
  Successfully uninstalled gensim-4.3.3
Found existing installation: pandas 2.2.2
Uninstalling pandas-2.2.2:
  Successfully uninstalled pandas-2.2.2
  Using cached numpy-2.2.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
  Using cached gensim-4.3.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (8.1 kB)
  Using cached pandas-2.2.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (19 kB)
  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached pandas-2.2.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (13.0 MB)
Using cached gensim-4.3.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (26.7 MB)
Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylin

In [2]:
import pandas as pd

In [3]:
import os
filename = 'dataset.txt'
text_path = 'https://raw.githubusercontent.com/jmestanza/natural-language-processing-practice/refs/heads/main/desafios/Desafio_2/the_adventures_of_sherlock_holmes.txt'
curr_dir = os.getcwd()
dataset_path = os.path.join(curr_dir, filename)
if not os.path.exists(dataset_path):
  !wget {text_path} -O {filename}

os.listdir(curr_dir)


['.config',
 'dataset_nltk_preprocessed.txt',
 'dataset_clean.txt',
 'dataset_clean_preprocessed.txt',
 'dataset.txt',
 'sample_data']

In [4]:
with open(dataset_path, 'r') as f:
  text = f.read()

print(text[:1500])

﻿The Project Gutenberg eBook of The Adventures of Sherlock Holmes
    
This ebook is for the use of anyone anywhere in the United States and
most other parts of the world at no cost and with almost no restrictions
whatsoever. You may copy it, give it away or re-use it under the terms
of the Project Gutenberg License included with this ebook or online
at www.gutenberg.org. If you are not located in the United States,
you will have to check the laws of the country where you are located
before using this eBook.

Title: The Adventures of Sherlock Holmes

Author: Arthur Conan Doyle

Release date: March 1, 1999 [eBook #1661]
                Most recently updated: October 10, 2023

Language: English

Credits: an anonymous Project Gutenberg volunteer and Jose Menendez


*** START OF THE PROJECT GUTENBERG EBOOK THE ADVENTURES OF SHERLOCK HOLMES ***




The Adventures of Sherlock Holmes

by Arthur Conan Doyle


Contents

   I.     A Scandal in Bohemia
   II.    The Red-Headed League
   III.   A 

## Limpieza de dataset
Debemos obtener solo los textos de adentro de los capítulos y subcapítulos.

Además debemos ignorar el texto que viene con la metadata de la obra (presente en el principio y final del dataset).

La estructura es


```
I. Chapter title
I.
text
II.
text
```



In [5]:
import re

# Read the text file
with open(dataset_path, "r", encoding="utf-8") as file:
    text = file.read()

# Remove everything before the actual content
start_marker = "*** START OF THE PROJECT GUTENBERG EBOOK"
end_marker = "*** END OF THE PROJECT GUTENBERG EBOOK"

# Keep only the content between the start and end markers
text = text.split(start_marker, 1)[-1].split(end_marker, 1)[0]

# Define a pattern to match chapter titles (Roman numerals followed by title)
chapter_pattern = re.compile(r"\n([IVXLCDM]+)\.\s([^\n]+)\n")

# Find all matches (chapter titles)
matches = list(chapter_pattern.finditer(text))

# Split the text into chapters
chapters = []
start = matches[0].end() if matches else 0  # Start at first chapter if found
chapter_contents = []

for i in range(len(matches)):
    end = matches[i + 1].start() if i + 1 < len(matches) else len(text)

    chapter_title = matches[i].group(2)  # Extract chapter title (ignore Roman numeral)
    chapter_content = text[matches[i].end():end].strip()

    # Remove subchapter numbers (standalone Roman numerals followed by a newline)
    chapter_content = re.sub(r"^\n?[IVXLCDM]+\.\n+", "", chapter_content, flags=re.MULTILINE).strip()

    chapters.append(chapter_title)
    # adding title as part of the content
    chapter_contents.append(f"{chapter_title}\n{chapter_content}")

df_text = pd.DataFrame({'chapter': chapters, 'content': chapter_contents})
df_text.head()

,chapter,content
0,A SCANDAL IN BOHEMIA,A SCANDAL IN BOHEMIA\nTo Sherlock Holmes she i...
1,THE RED-HEADED LEAGUE,THE RED-HEADED LEAGUE\nI had called upon my fr...
2,A CASE OF IDENTITY,"A CASE OF IDENTITY\n“My dear fellow,” said She..."
3,THE BOSCOMBE VALLEY MYSTERY,THE BOSCOMBE VALLEY MYSTERY\nWe were seated at...
4,THE FIVE ORANGE PIPS,THE FIVE ORANGE PIPS\nWhen I glance over my no...


In [6]:
new_text = "\n".join(df_text["content"].tolist())
print(new_text[:500])

A SCANDAL IN BOHEMIA
To Sherlock Holmes she is always _the_ woman. I have seldom heard him
mention her under any other name. In his eyes she eclipses and
predominates the whole of her sex. It was not that he felt any emotion
akin to love for Irene Adler. All emotions, and that one particularly,
were abhorrent to his cold, precise but admirably balanced mind. He
was, I take it, the most perfect reasoning and observing machine that
the world has seen, but as a lover he would have placed himself in


In [7]:
with open('dataset_clean.txt', 'w') as f:
  f.write(new_text)

Observando el dataset limpio, vemos que
- `\n\n` (consecutive endline)
- `_the_` (emphasis)
- `“Seven!”` (punctuation)
- `“Absolutely?”` (punctuation)
- `John’s` (punctuation)
- `won’t` (contractions)
- `half-dragged` (hyphened compound-words)
- `“‘The Church of St. Monica,` (abbreviation)
- `\t\t“MY DEAR MR. SHERLOCK HOLMES` (tabs/spaces)
- `_née_` (tilde)
- `“‘Never.’` (quotes)
- `“THE RED-HEADED LEAGUE IS DISSOLVED. October 9, 1890.”` (numbered date)
- `sitting-room`
- `“‘December 22nd. Twenty-four geese at 7_s_. 6_d_.’”`
- `One night—it was on the twentieth of March, 1888—`  (em dash: longer and generally used to note a break, provide drama, or give an example)
- ` to a salary of £ 4 a` (Simbols & numbers)

Deberíamos aplicar un preprocesamiento al texto. Un ejemplo con nltk es el siguiente:

In [8]:
import re
import contractions
import dateparser
import string

def preprocess_sherlock(text):
    text = text.lower()

    # 1. Normalize Line Breaks & Whitespace
    text = re.sub(r'\n+', '\n', text)  # Reduce multiple newlines to one
    text = re.sub(r'[ \t]+', ' ', text)  # Replace multiple spaces/tabs with a single space


    # 2. Remove Emphasis (_the_ → the)
    text = re.sub(r'_([^_]+)_', r'\1', text)

    # 3. Expand Contractions (won’t → will not)
    text = contractions.fix(text)

    # 4. Standardize Quotes & Punctuation
    text = re.sub(r"[“”]", '"', text)  # Double quotes
    text = re.sub(r"[‘’]", "'", text)  # Single quotes
    # remove all quotes
    text = re.sub(r'["]', '', text)
    text = re.sub(r'[\']', '', text)


    # 5. Normalize Hyphenated Words (half-dragged → half dragged)
    # matches both hyphens (-) and em dashes (—)
    text = re.sub(r'—', r' ', text)
    text = re.sub(r'-', r' ', text)

    return text


In [9]:
with open('dataset_clean_preprocessed.txt', 'w') as f:
  preprocessed_text = preprocess_sherlock(new_text)
  f.write(preprocessed_text)

In [10]:
print(preprocessed_text[:1500])

a scandal in bohemia
to sherlock holmes she is always the woman. i have seldom heard him
mention her under any other name. in his eyes she eclipses and
predominates the whole of her sex. it was not that he felt any emotion
akin to love for irene adler. all emotions, and that one particularly,
were abhorrent to his cold, precise but admirably balanced mind. he
was, i take it, the most perfect reasoning and observing machine that
the world has seen, but as a lover he would have placed himself in a
false position. he never spoke of the softer passions, save with a gibe
and a sneer. they were admirable things for the observer excellent for
drawing the veil from mens motives and actions. but for the trained
reasoner to admit such intrusions into his own delicate and finely
adjusted temperament was to introduce a distracting factor which might
throw a doubt upon all his mental results. grit in a sensitive
instrument, or a crack in one of his own high power lenses, would not
be more disturbin

In [11]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string

# Descargar tokenizador punkt
nltk.download('punkt_tab')
nltk.download("punkt")
# Descargar la red de semántica del inglés WordNet
# Es una extensa red semántica que puede usarse (entre otras cosas)
# para hacer POS tagging o lematizar.
nltk.download("wordnet")
# Descargar diccionario de stopwords del inglés
nltk.download('stopwords')
# Para usar NLTK 3.6.6 o superior es necesario instalar OMW 1.4
# (Open Multilingual WordNet)
nltk.download('omw-1.4')


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [12]:
def nltk_process(text, debug=False):
    # Tokenization
    nltk_tokenList = word_tokenize(text)

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    nltk_lemmaList = [lemmatizer.lemmatize(word, pos='v') for word in nltk_tokenList]  # Specify 'v' for verbs (was turning 'was' to 'wa')


    if debug:
      print("Lemmatization")
      print(nltk_lemmaList[:50])

    # Stop words
    nltk_stop_words = set(stopwords.words("english"))
    filtered_sentence = [w for w in nltk_lemmaList if w not in nltk_stop_words]

    # # Filter Punctuation <-- ya filtrado
    filtered_sentence = [w for w in filtered_sentence if w not in string.punctuation]

    if debug:
      print(" ")
      print("Remove stopword & Punctuation")
      print(filtered_sentence[:50])
    return " ".join(filtered_sentence)

In [13]:
with open('dataset_nltk_preprocessed.txt', 'w') as f:
  nltk_preprocessed_text = nltk_process(preprocessed_text, debug=True)
  f.write(nltk_preprocessed_text)

Lemmatization
['a', 'scandal', 'in', 'bohemia', 'to', 'sherlock', 'holmes', 'she', 'be', 'always', 'the', 'woman', '.', 'i', 'have', 'seldom', 'hear', 'him', 'mention', 'her', 'under', 'any', 'other', 'name', '.', 'in', 'his', 'eye', 'she', 'eclipse', 'and', 'predominate', 'the', 'whole', 'of', 'her', 'sex', '.', 'it', 'be', 'not', 'that', 'he', 'felt', 'any', 'emotion', 'akin', 'to', 'love', 'for']
 
Remove stopword & Punctuation
['scandal', 'bohemia', 'sherlock', 'holmes', 'always', 'woman', 'seldom', 'hear', 'mention', 'name', 'eye', 'eclipse', 'predominate', 'whole', 'sex', 'felt', 'emotion', 'akin', 'love', 'irene', 'adler', 'emotions', 'one', 'particularly', 'abhorrent', 'cold', 'precise', 'admirably', 'balance', 'mind', 'take', 'perfect', 'reason', 'observe', 'machine', 'world', 'see', 'lover', 'would', 'place', 'false', 'position', 'never', 'speak', 'softer', 'passions', 'save', 'gibe', 'sneer', 'admirable']


## 1 - Preprocesamiento

In [14]:
df_text["content"] = df_text["content"].apply(preprocess_sherlock)
df_text["content"] = df_text["content"].apply(nltk_process)
df_text.head()

,chapter,content
0,A SCANDAL IN BOHEMIA,scandal bohemia sherlock holmes always woman s...
1,THE RED-HEADED LEAGUE,red head league call upon friend mr. sherlock ...
2,A CASE OF IDENTITY,case identity dear fellow say sherlock holmes ...
3,THE BOSCOMBE VALLEY MYSTERY,boscombe valley mystery seat breakfast one mor...
4,THE FIVE ORANGE PIPS,five orange pip glance note record sherlock ho...


In [15]:
print("Cantidad de documentos:", df_text.shape[0])

Cantidad de documentos: 12


In [16]:
from tensorflow.keras.preprocessing.text import text_to_word_sequence

doc_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df_text.iterrows():
    doc_tokens.append(text_to_word_sequence(row["content"]))



In [17]:
for doc_token in doc_tokens:
  print(doc_token[:10])
# sentence_tokens[:100]

['scandal', 'bohemia', 'sherlock', 'holmes', 'always', 'woman', 'seldom', 'hear', 'mention', 'name']
['red', 'head', 'league', 'call', 'upon', 'friend', 'mr', 'sherlock', 'holmes', 'one']
['case', 'identity', 'dear', 'fellow', 'say', 'sherlock', 'holmes', 'sit', 'either', 'side']
['boscombe', 'valley', 'mystery', 'seat', 'breakfast', 'one', 'morning', 'wife', 'maid', 'bring']
['five', 'orange', 'pip', 'glance', 'note', 'record', 'sherlock', 'holmes', 'case', 'years']
['man', 'twist', 'lip', 'isa', 'whitney', 'brother', 'late', 'elias', 'whitney', 'd']
['adventure', 'blue', 'carbuncle', 'call', 'upon', 'friend', 'sherlock', 'holmes', 'upon', 'second']
['adventure', 'speckle', 'band', 'glance', 'note', 'seventy', 'odd', 'case', 'last', 'eight']
['adventure', 'engineer', 'thumb', 'problems', 'submit', 'friend', 'mr', 'sherlock', 'holmes', 'solution']
['adventure', 'noble', 'bachelor', 'lord', 'st', 'simon', 'marriage', 'curious', 'termination', 'long']
['adventure', 'beryl', 'coronet', 'h

### 2 - Crear los vectores (word2vec)

In [18]:
from gensim.models.callbacks import CallbackAny2Vec

class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        # else:
        if self.epoch != 0 and self.epoch % 10 == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [19]:
from gensim.models import Word2Vec
# Crearmos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     vector_size=300,       # dimensionalidad de los vectores
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

In [20]:
# Obtener el vocabulario con los tokens
w2v_model.build_vocab(doc_tokens)

In [21]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 12


In [22]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.index_to_key))

Cantidad de words distintas en el corpus: 1761


3 - Entrenar embeddings

In [23]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(doc_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=200,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 711945.25
Loss after epoch 10: 315665.25
Loss after epoch 20: 257323.0
Loss after epoch 30: 216560.0
Loss after epoch 40: 205909.0
Loss after epoch 50: 201692.0
Loss after epoch 60: 198725.0
Loss after epoch 70: 170026.0
Loss after epoch 80: 169006.0
Loss after epoch 90: 166178.0
Loss after epoch 100: 163550.0
Loss after epoch 110: 161976.0
Loss after epoch 120: 159618.0
Loss after epoch 130: 157672.0
Loss after epoch 140: 155604.0
Loss after epoch 150: 152702.0
Loss after epoch 160: 150108.0
Loss after epoch 170: 146982.0
Loss after epoch 180: 97384.0
Loss after epoch 190: 93740.0


(7156592, 9424400)

### 4 - Ensayar

In [29]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["holmes"], topn=10)

[('sherlock', 0.31480851769447327),
 ('duncan', 0.2673177421092987),
 ('mr', 0.2506963908672333),
 ('tut', 0.24674837291240692),
 ('leadenhall', 0.24650242924690247),
 ('narrative', 0.24618887901306152),
 ('found', 0.24282191693782806),
 ('speckle', 0.23919610679149628),
 ('neville', 0.23918063938617706),
 ('eventually', 0.23867033421993256)]

In [30]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["sherlock"], topn=10)

[('holmes', 0.31480854749679565),
 ('custom', 0.30555611848831177),
 ('inn', 0.2916117012500763),
 ('associate', 0.28934550285339355),
 ('pen', 0.28260719776153564),
 ('art', 0.277919739484787),
 ('fold', 0.2764796018600464),
 ('offices', 0.27618879079818726),
 ('rule', 0.2747381627559662),
 ('lysander', 0.27125275135040283)]

In [31]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["watson"], topn=10)

[('scoundrel', 0.3240799307823181),
 ('roylott', 0.30476751923561096),
 ('nervous', 0.28471583127975464),
 ('supper', 0.28310444951057434),
 ('things', 0.2802175283432007),
 ('niece', 0.27739405632019043),
 ('stoner', 0.2746654152870178),
 ('grimesby', 0.27122586965560913),
 ('glad', 0.2665504515171051),
 ('impossible', 0.26202866435050964)]

In [32]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["detective"], topn=10)

[('stoper', 0.34211307764053345),
 ('sideboard', 0.3415271043777466),
 ('solution', 0.336597740650177),
 ('official', 0.33461782336235046),
 ('goodness', 0.3319486379623413),
 ('swandam', 0.32684096693992615),
 ('pain', 0.32170018553733826),
 ('slip', 0.32121652364730835),
 ('neck', 0.31688380241394043),
 ('nervous', 0.3085125982761383)]

In [33]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["case"], topn=10)

[('innocence', 0.30823269486427307),
 ('incline', 0.2733447551727295),
 ('spite', 0.27135947346687317),
 ('trivial', 0.2701475918292999),
 ('conviction', 0.2697224020957947),
 ('forever', 0.268420934677124),
 ('1000', 0.2684140205383301),
 ('postmark', 0.2674146294593811),
 ('visitors', 0.2671177089214325),
 ('ones', 0.26599249243736267)]